In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q accelerate==0.21.0  transformers==4.31.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [20]:
import pandas as pd
dataset = pd.read_excel("./drive/MyDrive/positive_negative/data.xlsx")

In [21]:
dataset.head()

,Sentence_from_X,Matched_Sentence_from_Y
0,When I lived in Los Angeles a year and a half ...,This place features everything you can expect ...
1,"I've always liked PF Chang's and, generally sp...",Right off the bat I want to admit that when it...
2,This is my 2nd time here and I thought that I ...,What is it about the anonymous Chinese buffet ...
3,"I love Whole Foods, and I have been a fan for ...",I am a big fan of the Whole Foods company in g...
4,TOTALLY one of the best restaurants in Pittsb...,"I wasn't really a fan. In fact, this may hae b..."


In [22]:
len(dataset)

893

In [23]:
dataset["Matched_Sentence_from_Y"][3]

"I am a big fan of the Whole Foods company in general.  The store back home (Omaha) had a wine bar, espresso bar, gelato, sushi and udon stand, a GIANT cheese counter where you could sample...I could go on.\\n\\nThe Pittsburgh Whole Foods is small, and almost as overcrowded as its parking lot.  Two-thirds of the time they are out of what I came for.  I'm sorry, but if I'm paying THAT much for groceries, I want a stellar shopping experience.  This WF is mediocre at best."

In [24]:
# Identify rows with NaN values
rows_with_nan = dataset[dataset.isna().any(axis=1)]

# Print rows with NaN values
print("Rows with NaN values:")
print(rows_with_nan)

Rows with NaN values:
    Sentence_from_X                            Matched_Sentence_from_Y
285             NaN  [3 Stars by PHX comparison and 2 overall] \n\n...


In [25]:
dataset = dataset.dropna()

In [26]:
# Identify rows with NaN values
rows_with_nan = dataset[dataset.isna().any(axis=1)]

# Print rows with NaN values
print("Rows with NaN values:")
print(rows_with_nan)

Rows with NaN values:
Empty DataFrame
Columns: [Sentence_from_X, Matched_Sentence_from_Y]
Index: []


In [27]:
import unicodedata
import re
import numpy as np
import os
import io
import time
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  if type(w) != str:
    print(w)
    return ""

  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿:;])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)
  w = re.sub(r'\\n'," ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",","¿")
  w = re.sub(r"[^a-zA-Z?.!,¿:;]+", " ", w)

    # remove extra space
  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  return w

In [ ]:
print(preprocess_sentence(dataset["Sentence_from_X"][3]))

i am taking away a star because they dont deliver if china taste delivered i could easily see myself eating from here weekly this is a hole in the wall in a seedy neighborhood but i just cant get enoughnnthe chinese food is some of the best ive had typically get the hunan shrimp or schezwan shrimp with a side of pork fried rice the shrimp are big and taste fresh not frozen large and cooked perfectly and the veggies are always crisp and not soggy and over done also weve had the sweet and sour chicken and general tsos chicken nalthough the restaurant is tiny and a little run down the food is very goodnall the entrees ive ever had were served pipping hot nnthe restaurant is really nothing pretty and there are a total of tables the restaurant is tiny if youre passing through the northside check this place out


In [ ]:
print(preprocess_sentence(dataset["Matched_Sentence_from_Y"][3]))

right off the bat i want to admit that when it comes to chinese food im spoiled ive traveled to asia number of times eaten a lot in san francisco and explored the chinese restaurants in phoenix for a lot of years i have learned to expect a more authentic type of chinese foodthe type of place that is frequented by actual chinese peoplennstill i love these little family run places so we checked this place out hoping for the best i wanted to like it but i just did not like the food the crispy chicken chow mien noodles were the type you get out of a can or bag not the real thing the other items were pretty standard chineseamerican fare honestly i like panda better and it is cheaper my stepson loved the shrimp thoughsaid it was the best he has had since living in the us so your mileage may vary nnothers have enjoyed this place the service is friendly and they deliver if you live in the general area portions are very large if you are not expecting a lot you might like it better


In [ ]:
print(preprocess_sentence(dataset["Matched_Sentence_from_Y"][3]).encode("UTF-8"))

b'right off the bat i want to admit that when it comes to chinese food im spoiled ive traveled to asia number of times eaten a lot in san francisco and explored the chinese restaurants in phoenix for a lot of years i have learned to expect a more authentic type of chinese foodthe type of place that is frequented by actual chinese peoplennstill i love these little family run places so we checked this place out hoping for the best i wanted to like it but i just did not like the food the crispy chicken chow mien noodles were the type you get out of a can or bag not the real thing the other items were pretty standard chineseamerican fare honestly i like panda better and it is cheaper my stepson loved the shrimp thoughsaid it was the best he has had since living in the us so your mileage may vary nnothers have enjoyed this place the service is friendly and they deliver if you live in the general area portions are very large if you are not expecting a lot you might like it better'


In [28]:
def create_dataset(data):
  positive_sent = [preprocess_sentence(positive) for positive in data['Sentence_from_X']]
  negative_sent = [preprocess_sentence(negative) for negative in data['Matched_Sentence_from_Y']]
  return positive_sent,negative_sent
positive_sent, negative_sent = create_dataset(dataset)

In [29]:
from sklearn.model_selection import train_test_split

# Load model directly
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
import torch

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")
train_inputs, test_inputs, train_targets, test_targets = train_test_split(positive_sent, negative_sent, test_size=0.2, random_state=42)


In [30]:
max_source_length = max(len(og) for og in positive_sent)
max_target_length = max(len(t) for t in negative_sent)
print(max_source_length,max_target_length)

4076 3862


In [31]:
a = ""
l = 0
for og in positive_sent:
  if len(og)>l:
    l = len(og)
    a = og
print(l)
print(og)

4076
always see this restaurant when i m grocery shopping at smiths . curiosity kicked in and wanted to try . outside appearance was not appealing . i almost didn t go in . but before i was gona leave i decided to depend on my yelp friends . it seemed to have some good reviews . first impression was divey . really needs some help . decor is old and the place is small . however the lady was very nice . i ordered chicken pad thai from lunch special . the menu says you get soup . then she nicely tells me you only get soup if you done in . she was nice and offered me done while i wait for my food . maybe she heard my stomach growl . lol . . my friend ordered orange chicken . also from lunch special . was happy when i got the bill . it was for both meals . now that s what i call a lunch special when i got home . i noticed they gave decent proportion sizes . my chicken pad thai was delicious . my friends orange chicken was good but the sauce was more runny than what i m use to seeing . didn 

In [32]:
class TranslationDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, inputs, targets):
        self.tokenizer = tokenizer
        self.input_texts = inputs
        self.target_texts = targets
        self.max_seq_length = 512  # Maximum sequence length

    def __len__(self):
        return len(self.input_texts)

    def __getitem__(self, idx):
        input_text = self.input_texts[idx]
        target_text = self.target_texts[idx]
        task_prefix = "Change Positive Sentence to Negative Sentence: "

        # Tokenize input text
        inputs = self.tokenizer(
            task_prefix + input_text,
            padding='max_length',
            truncation=True,
            max_length=self.max_seq_length,
            return_tensors='pt',
        )

        # Tokenize target text
        labels = self.tokenizer(
            target_text,
            padding='max_length',
            truncation=True,
            max_length=self.max_seq_length,
            return_tensors='pt',
        )

        # Extract input_ids and attention_mask
        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()

        # Extract labels
        labels = labels['input_ids'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

train_dataset = TranslationDataset(tokenizer,train_inputs,train_targets)
test_dataset = TranslationDataset(tokenizer,test_inputs,test_targets)

In [ ]:
print(train_dataset.__getitem__)

<bound method TranslationDataset.__getitem__ of <__main__.TranslationDataset object at 0x7d2b5e7c50f0>>


In [ ]:
print("Maximum tokenizer length:", tokenizer.model_max_length)

Maximum tokenizer length: 512


In [33]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [34]:
from transformers import EarlyStoppingCallback
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=50,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    evaluation_strategy='steps',  # Evaluate every `eval_steps` steps
    eval_steps=100,
    fp16=True
)

# Instantiate Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)
# Train the model in training mode
model.train()
# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
100,No log,1.370275
200,No log,1.159462
300,No log,1.109589
400,No log,1.084911
500,2.559200,1.069068
600,2.559200,1.053462
700,2.559200,1.042703
800,2.559200,1.030125
900,2.559200,1.020414
1000,1.099000,1.011519


TrainOutput(global_step=4500, training_loss=1.1500564303927951, metrics={'train_runtime': 1880.0776, 'train_samples_per_second': 18.962, 'train_steps_per_second': 2.394, 'total_flos': 4824935222476800.0, 'train_loss': 1.1500564303927951, 'epoch': 50.0})

In [35]:
model_save_path = './drive/My Drive/positive_negative/t5-pos_neg'

# Make sure to create the directory if it doesn't exist
import os
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

# Now save the model
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

('./drive/My Drive/positive_negative/t5-pos_neg/tokenizer_config.json',
 './drive/My Drive/positive_negative/t5-pos_neg/special_tokens_map.json',
 './drive/My Drive/positive_negative/t5-pos_neg/spiece.model',
 './drive/My Drive/positive_negative/t5-pos_neg/added_tokens.json')

In [44]:
print("--------------Iteration 1----------")
pos_sent=["I love the sushi at this place.","pizza is pretty good ! the staff is very nice , and good service ."]
model.eval()
for i in pos_sent:
  task_prefix = "Change Positive Sentence to Negative Sentence: "
  input_ids = tokenizer(
      task_prefix + i, max_length=512, padding=True, truncation=True, return_tensors="pt"
  )  # Batch size 1
  # Move tensors to the GPU if available
  input_ids = input_ids.input_ids.to(device)
  outputs = model.generate(input_ids=input_ids)
  print("Positive Input---",i)
  print("Negative Output---",tokenizer.decode(outputs[0], skip_special_tokens=True))

--------------Iteration 1----------
Positive Input--- I love the sushi at this place.
Negative Output--- i love sushi and the food here. the sushi is mediocre. the
Positive Input--- pizza is pretty good ! the staff is very nice , and good service .
Negative Output--- pizza is good, but not good. service good, but not good.


In [40]:
positive_sent

['when i lived in los angeles a year and a half ago , p f chang s was no great shakes , one of millions of places with good but not outstanding stuff on the menu . here in pittsburgh , it s the only chinese place i ve found that combines genuinely good service with quality food . not , perhaps , as good as what you can get in la s top places , but fantastic compared to its competition here . the kung pao chicken is particularly good , with delicious fried chicken cubes and the usual hot sauce and chiles . chicken with lettuce wraps is good too . their fried noodle dish with chicken , whose name i have forgotten , is addictive but has an odd sauce i don t quite like . still , they get the friedness of the noodles right , and there are precious few . . . ahem . . . no other restaurants around here that do . portions are small , so either plan to get an appetizer and an entree or expect to leave almost as hungry as you came . the minced chicken in lettuce wraps appetizer is outstanding an

In [51]:
import gensim.downloader as api
glove_model = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [53]:
!pip install POT

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [52]:
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import preprocess_string

def preprocess(text):
    # Simple preprocessing: tokenize, lower, alpha characters
    return preprocess_string(text)

def compute_wmd(sentence_a, sentence_b):
    preprocessed_a = preprocess(sentence_a)
    preprocessed_b = preprocess(sentence_b)
    return glove_model.wmdistance(preprocessed_a, preprocessed_b)


# Placeholder for the translation model function
def translate(sentence):
    model.eval()
    task_prefix = "Change Positive Sentence to Negative Sentence: "
    input_ids = tokenizer(task_prefix + sentence, max_length=512, padding=True, truncation=True, return_tensors="pt")
    input_ids = input_ids.input_ids.to(device)
    outputs = model.generate(input_ids=input_ids)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

updated_pairs = []
counter = 0
for index, row in dataset.iterrows():
    positive_sentence = row['Sentence_from_X']
    matched_negative_sentence = row['Matched_Sentence_from_Y']

    # Generate translation
    model_negative_sentence = translate(positive_sentence)

    # Compute WMD between original pair and translation to matched_sentence
    original_wmd = compute_wmd(positive_sentence, matched_negative_sentence)
    translated_wmd = compute_wmd(positive_sentence, model_negative_sentence)

    # If the translated sentence is closer to the matched_sentence, update the pair
    if translated_wmd < original_wmd:
        counter = counter + 1
        updated_pairs.append((positive_sentence, model_negative_sentence))
    else:
        updated_pairs.append((positive_sentence, matched_negative_sentence))

# Create a new DataFrame with updated pairs
df_updated_matches = pd.DataFrame(updated_pairs, columns=['Sentence_from_X', 'Matched_Sentence_from_Y'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


ModuleNotFoundError: No module named 'ot'